In [201]:
import numpy as np 
import pandas as pd
import datetime as dt
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances,manhattan_distances
from scipy.sparse import csr_matrix,hstack
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy


In [75]:
df=pd.read_csv('/Users/shivanshgupta/Desktop/Research shit/archive-2/movies_metadata.csv',low_memory=False)
rating=pd.read_csv('/Users/shivanshgupta/Desktop/Research shit/archive-2/ratings_small.csv',low_memory=False)
ids=pd.read_csv('/Users/shivanshgupta/Desktop/Research shit/archive-2/links_small.csv',low_memory=False)

In [76]:
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [77]:
ids.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [78]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [80]:
df.isnull().sum(axis=0).sort_values(ascending=False)

belongs_to_collection    40972
homepage                 37684
tagline                  25054
overview                   954
poster_path                386
runtime                    263
status                      87
release_date                87
imdb_id                     17
original_language           11
spoken_languages             6
title                        6
video                        6
vote_average                 6
revenue                      6
vote_count                   6
popularity                   5
production_companies         3
production_countries         3
original_title               0
id                           0
genres                       0
budget                       0
adult                        0
dtype: int64

In [81]:
df=df[['genres','overview','title','id','imdb_id','vote_average','vote_count','release_date']]

In [82]:
df=df.dropna(subset=['imdb_id','release_date','title','overview'],axis=0)


In [83]:
df.isnull().sum(axis=0).sort_values(ascending=False)

genres          0
overview        0
title           0
id              0
imdb_id         0
vote_average    0
vote_count      0
release_date    0
dtype: int64

In [84]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
vote_average,44423.0,5.644162,1.888368,0.0,5.0,6.0,6.8,10.0
vote_count,44423.0,112.267181,496.752403,0.0,3.0,10.0,35.0,14075.0


In [85]:
df['imdb_id']=df['imdb_id'].str.extract(r'(\d+)').astype('int').astype('str')

In [86]:
df.imdb_id.head()

0    114709
1    113497
2    113228
3    114885
4    113041
Name: imdb_id, dtype: object

In [87]:
df.head()

,genres,overview,title,id,imdb_id,vote_average,vote_count,release_date
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",Toy Story,862,114709,7.7,5415.0,1995-10-30
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,Jumanji,8844,113497,6.9,2413.0,1995-12-15
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,Grumpier Old Men,15602,113228,6.5,92.0,1995-12-22
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,31357,114885,6.1,34.0,1995-12-22
4,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,Father of the Bride Part II,11862,113041,5.7,173.0,1995-02-10


In [88]:
df['release_date'].head()

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object

In [89]:
df['release_date'].dtype

dtype('O')

In [90]:
df['release_date']=pd.to_datetime(df.release_date,format='%Y-%m-%d')

In [91]:
df.insert(loc=5,column='year',value=df.release_date.dt.year)

In [92]:
df.year.head()

0    1995
1    1995
2    1995
3    1995
4    1995
Name: year, dtype: int32

In [93]:
df=df.drop('release_date',axis=1)

In [94]:
df.iloc[100].genres

"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'name': 'Mystery'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 53, 'name': 'Thriller'}]"

In [95]:
df.genres=df.genres.apply(literal_eval)

In [96]:
df.genres.head()

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object

In [97]:
df['genres']=df.genres.apply(lambda x:[i['name'].lower() for i in x] if isinstance(x,list) else [])

In [98]:
df.vote_count

0        5415.0
1        2413.0
2          92.0
3          34.0
4         173.0
          ...  
45460      26.0
45462       3.0
45463       6.0
45464       0.0
45465       0.0
Name: vote_count, Length: 44423, dtype: float64

In [99]:
a=df[df['vote_average']>9]['vote_average'].value_counts()
print(a.head())#vote average is mildly impotent as it favors movies with higher vote count 

vote_average
10.0    179
9.5      18
9.3      16
9.2       4
9.4       3
Name: count, dtype: int64


In [100]:
b = df.groupby(df['vote_average'] > 9).apply(lambda x: (x['vote_count'], x.index))
print(b)

vote_average
False    ([5415.0, 2413.0, 92.0, 34.0, 173.0, 1886.0, 1...
True     ([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0,...
dtype: object


In [101]:
#As forth weighted rating is opted to solve this problem
'''WR=(v/v+m)*r+(m/m+v)*c'''

'WR=(v/v+m)*r+(m/m+v)*c'

In [102]:
c=df['vote_average'].mean()
m=df.vote_count.quantile(0.90)

In [103]:
def WR(x,C=c,m=m):
    v = x['vote_count']
    R = x['vote_average']
    return (v / (v + m) * R) + (m / (v + m) * C)

In [104]:
df.insert(loc=1,column='Weighted_Rating',value=WR(df))

In [105]:
df.Weighted_Rating

0        7.639209
1        6.819622
2        5.950532
3        5.722044
4        5.672742
           ...   
45460    5.651763
45462    5.704087
45463    5.579454
45464    5.644162
45465    5.644162
Name: Weighted_Rating, Length: 44423, dtype: float64

In [106]:
df = df.rename(columns={'imdb_id': 'imdbId'})
df.head()

,genres,Weighted_Rating,overview,title,id,imdbId,year,vote_average,vote_count
0,"[animation, comedy, family]",7.639209,"Led by Woody, Andy's toys live happily in his ...",Toy Story,862,114709,1995,7.7,5415.0
1,"[adventure, fantasy, family]",6.819622,When siblings Judy and Peter discover an encha...,Jumanji,8844,113497,1995,6.9,2413.0
2,"[romance, comedy]",5.950532,A family wedding reignites the ancient feud be...,Grumpier Old Men,15602,113228,1995,6.5,92.0
3,"[comedy, drama, romance]",5.722044,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,31357,114885,1995,6.1,34.0
4,[comedy],5.672742,Just when George Banks has recovered from his ...,Father of the Bride Part II,11862,113041,1995,5.7,173.0


In [107]:
df.overview.iloc[10]

"Widowed U.S. president Andrew Shepherd, one of the world's most powerful men, can have anything he wants -- and what he covets most is Sydney Ellen Wade, a Washington lobbyist. But Shepherd's attempts at courting her spark wild rumors and decimate his approval ratings."

In [108]:
genres_one_hot = df['genres'].str.join('|').str.get_dummies()


In [109]:
genre_matrix=csr_matrix(genres_one_hot)

In [110]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])
tfidf_matrix=csr_matrix(tfidf_matrix)

In [111]:
num_neighbors = 5

In [112]:
year_encoder = OneHotEncoder(sparse_output=True)
year_one_hot = year_encoder.fit_transform(df[['year']])

In [136]:
rating_ohe=year_encoder.fit_transform(df[['Weighted_Rating']])

In [137]:
combined_features = hstack([csr_matrix(genres_one_hot), tfidf_matrix, year_one_hot,rating_ohe])

In [138]:
nn = NearestNeighbors(n_neighbors=num_neighbors, metric='cosine').fit(combined_features)
distances, indices = nn.kneighbors(combined_features)

In [142]:
print(indices)

[[    0   719 26868 15273  2977]
 [    1    58 40031   177  9624]
 [    2    50   176   273    66]
 ...
 [44420  9358  9429  9367  9352]
 [44421 27455 32956 41129 42179]
 [44422 41974 41883 43994 41589]]


In [139]:
def content_recommendations(title):

    title = title.lower()
    df['title'] = df['title'].str.lower()
    
    idx = df[df['title'] == title].index
    
    if len(idx) == 0:  
        return "Title not found"
    
    idx = idx[0]  
    
    neighbor_indices = indices[idx]
    
    return df['title'].iloc[neighbor_indices]


In [140]:
content_recommendations('La la land')

41751    perry mason: the case of the scandalous scoundrel
25435                       flower and snake 5: rope magic
36916                     never travel on a one way ticket
20717                                    business as usual
23101                                   season of monsters
Name: title, dtype: object

In [117]:
ratings=rating.merge(ids,how='inner',on='movieId')

In [118]:
ratings.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,1,31,2.5,1260759144,112792,9909.0
1,1,1029,3.0,1260759179,33563,11360.0
2,1,1061,3.0,1260759182,117665,819.0
3,1,1129,2.0,1260759185,82340,1103.0
4,1,1172,4.0,1260759205,95765,11216.0


In [124]:
print(df.dtypes)

genres              object
Weighted_Rating    float64
overview            object
title               object
id                  object
imdbId              object
year                 int32
vote_average       float64
vote_count         float64
dtype: object


In [125]:
print(ratings.dtypes)

userId         int64
movieId        int64
rating       float64
timestamp      int64
imdbId         int64
tmdbId       float64
dtype: object


In [126]:
df.imdbId=df.imdbId.astype('int')

In [127]:
user=df.merge(ratings,how='inner',on='imdbId')

In [128]:
user.head()

,genres,Weighted_Rating,overview,title,id,imdbId,year,vote_average,vote_count,userId,movieId,rating,timestamp,tmdbId
0,"[animation, comedy, family]",7.639209,"Led by Woody, Andy's toys live happily in his ...",toy story,862,114709,1995,7.7,5415.0,7,1,3.0,851866703,862.0
1,"[animation, comedy, family]",7.639209,"Led by Woody, Andy's toys live happily in his ...",toy story,862,114709,1995,7.7,5415.0,9,1,4.0,938629179,862.0
2,"[animation, comedy, family]",7.639209,"Led by Woody, Andy's toys live happily in his ...",toy story,862,114709,1995,7.7,5415.0,13,1,5.0,1331380058,862.0
3,"[animation, comedy, family]",7.639209,"Led by Woody, Andy's toys live happily in his ...",toy story,862,114709,1995,7.7,5415.0,15,1,2.0,997938310,862.0
4,"[animation, comedy, family]",7.639209,"Led by Woody, Andy's toys live happily in his ...",toy story,862,114709,1995,7.7,5415.0,19,1,3.0,855190091,862.0


In [134]:
user=user.sort_values(by='userId')

In [133]:
user.columns

Index(['genres', 'Weighted_Rating', 'overview', 'title', 'id', 'imdbId',
       'year', 'vote_average', 'vote_count', 'userId', 'movieId', 'rating',
       'timestamp', 'tmdbId'],
      dtype='object')

In [143]:
user=user[['genres','title','userId','movieId','rating','timestamp']]

In [166]:
(user.title=="toy story").sum()

247

In [144]:
user.head()

,genres,title,userId,movieId,rating,timestamp
45677,"[science fiction, action, adventure]",tron,1,2105,4.0,1260759139
27693,"[drama, romance]",cinema paradiso,1,1172,4.0,1260759205
50538,"[horror, science fiction]",the fly,1,2455,2.5,1260759113
47977,"[adventure, animation, comedy, family]",antz,1,2294,2.0,1260759108
47047,"[adventure, drama, action]",willow,1,2193,2.0,1260759198


In [152]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user[['userId', 'movieId', 'rating']], reader)


In [154]:
trainset, testset = train_test_split(data, test_size=0.2,random_state=1000)


In [155]:
model = SVD()
model.fit(trainset)

In [188]:
def collaborative_recommendations(user_id, title, user, model, top_n=5):

    movie_row = user[user['title'] == title].drop_duplicates('movieId')
    
    if movie_row.empty:
        return f"Movie '{title}' not found."
    
    movie_id = movie_row['movieId'].values[0]

    all_movie_ids = user['movieId'].unique()
    other_movie_ids = [mid for mid in all_movie_ids if mid != movie_id]

    predictions = []

    for other_movie_id in other_movie_ids:
        pred = model.predict(user_id, other_movie_id)
        predictions.append((other_movie_id, pred.est))  

    predictions.sort(key=lambda x: x[1], reverse=True)
    
    recommended_ids = [pred[0] for pred in predictions][:top_n]
    
    recommended_titles = user[user['movieId'].isin(recommended_ids)]['title'].drop_duplicates().tolist()
    
    return recommended_titles

In [189]:
collaborative_recommendations(1,'antz',user,model)

['the usual suspects',
 'the maltese falcon',
 'the godfather',
 "one flew over the cuckoo's nest",
 'modern times']

In [209]:
def collaborative_recommendation(user_id, movie_id):
    pred = model.predict(user_id, movie_id)
    return pred.est  


In [218]:
def hybrid_recommendation(user_id, title, top_n=10):
       
    content_similar_movies = content_recommendations(title)
    
    if isinstance(content_similar_movies, str):
        return content_similar_movies  
    
    hybrid_scores = []

    for movie in content_similar_movies:
        movie_id = df[df['title'] == movie].iloc[0]['id']
        
        collab_score = float(collaborative_recommendation(user_id, movie_id))
        
        content_score = float(df[df['title'] == movie].iloc[0]['Weighted_Rating'])
        
        hybrid_score = round((0.5 * collab_score + 0.5 * content_score),3)
        
        hybrid_scores.append((movie, hybrid_score))
    
    hybrid_scores = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)
    
    return hybrid_scores[:top_n]


In [217]:
hybrid_recommendation(91,'antz')

[('i stand alone', 5.23),
 ('23', 5.06),
 ('beloved', 4.9),
 ('bloody angels', 4.9),
 ('black heaven', 4.9)]